<a href="https://colab.research.google.com/github/cfpinedao/MetNumUN2024II/blob/main/cfpinedao_Lab2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from io import BytesIO
import os
from urllib.parse import urlencode

import requests
import numpy as np
import pandas as pd


def read_avocadata(start_date, end_date, cache_loc='avocadata.html'):
    """Download avocado data to a dataframe.
    Parameters
    ----------
    """
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)
    base_url = 'https://www.marketnews.usda.gov/mnp/fv-report-retail'
    query_params = {
        'class': ['FRUITS'],
        'commodity': ['AVOCADOS'],
        'compareLy': ['No'],
        'endDate': [end_date.strftime("%m/%d/%Y")],
        'format': ['excel'],
        'organic': ['ALL'],
        'portal': ['fv'],
        'region': ['ALL'],
        'repDate': [start_date.strftime("%m/%d/%Y")],
        'type': ['retail'],
    }

    url = base_url + '?' + urlencode(query_params, doseq=1)

    if not os.path.exists(cache_loc):
        resp = requests.get(url, stream=True)
        resp.raise_for_status()

        with open(cache_loc, 'wb') as f:
            for block in resp.iter_content(chunk_size=4096):
                f.write(block)
        f.close()

    with open(cache_loc, 'rb') as f:
        frame = pd.read_html(f, header=0)[0]

    # Cleanup
    frame = frame[frame['Unit'] == 'each']
    frame['Organic'] = (frame['Organic'] == 'Y')
    frame['Variety'].replace(
        {'VARIOUS GREENSKIN VARIETIES': 'GREENSKIN'},
        inplace=True,
    )
    frame['Date'] = pd.to_datetime(frame['Date'].values, utc=True)

    frame['Region'] = frame['Region'].str.replace(' U.S.', '')
    frame['Region'] = frame['Region'].str.replace(' ', '_')

    # Drop useless columns.
    return frame.drop(
        ['Class', 'Commodity', 'Environment', 'Unit', '% Marked Local'],
        axis=1,
    )

In [2]:
## Needed Libraries.
from urllib.parse import urlencode
import pandas as pd

import requests

## In this case, we will obtain TOMATOES data.
base_url = 'https://www.marketnews.usda.gov/mnp/fv-report-retail'
query_params = {
  'class': ['VEGETABLES'], ## Change Fruit with Vegetables
  'commodity': ['TOMATOES'], ## Change Avocados with Tomatoes
  'compareLy': ['No'],
  'endDate': ['01/01/2021'], ## Hardcoding end Date
  'repDate': ['01/01/2020'], ## Hardcoding rep Date
  'format': ['excel'],
  'organic': ['ALL'],
  'portal': ['fv'],
  'region': ['ALL'],
  'type': ['retail'],
}

## Build URL to send request.
url = base_url + '?' + urlencode(query_params, doseq=1)

## Request data from url
resp = requests.get(url, stream=True)

## Create File with data.
with open('data.html', 'wb') as f:
            for block in resp.iter_content(chunk_size=4096):
                f.write(block)
            f.close()

## Read data from file.
tomatoes_data = pd.read_html('data.html', header=0)[0]
## Be careful, this data is uncleaned, If you want to use it correctly, you will first have to clean it


## You now have data of tomatoes in the United States! Hurra!

In [8]:
data

[{'nombre_del_acueducto': 'TONA BAJO',
  'vereda': 'TONA',
  'municipio': 'ALMEIDA',
  'provincia': 'ORIENTE',
  'corporacion': 'CORPOCHIVOR',
  'n_de_suscriptores': '18',
  'camara_de_comercio': 'No',
  'concesi_n_de_aguas': 'No',
  'fuente_abastecedora': 'NA'},
 {'nombre_del_acueducto': 'TONA ALTO',
  'vereda': 'TONA',
  'municipio': 'ALMEIDA',
  'provincia': 'ORIENTE',
  'corporacion': 'CORPOCHIVOR',
  'n_de_suscriptores': '48',
  'camara_de_comercio': 'Si',
  'concesi_n_de_aguas': 'Si',
  'fuente_abastecedora': 'NACIMIENTO MARTINEZ'},
 {'nombre_del_acueducto': 'ASOCIACIÓN DE SUSCRIPTORES ROSAL ALTO',
  'vereda': 'ROSAL',
  'municipio': 'ALMEIDA',
  'provincia': 'ORIENTE',
  'corporacion': 'CORPOCHIVOR',
  'n_de_suscriptores': '40',
  'camara_de_comercio': 'No',
  'concesi_n_de_aguas': 'No',
  'fuente_abastecedora': 'NA'},
 {'nombre_del_acueducto': 'EMPRESA DE SERVICIOS PÚBLICOS DE ALMEIDA ESPALMEIDA S.A. E.S.P - VEREDA ROSAL Y YAVIR',
  'vereda': 'ROSAL YAVIR',
  'municipio': 'ALME

gov


In [35]:
import requests
import pandas as pd
import plotly.graph_objects as go

url = "https://www.datos.gov.co/resource/2ir5-acis.json"
response = requests.get(url)

data = response.json()
df = pd.DataFrame(data)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df[col] for col in df.columns],
               fill_color='lavender',
               align='left'))
])
fig.update_layout(title="BASE DE DATOS ACUEDUCTOS RURALES DEPARTAMENTO DE BOYACÁ")
fig.show()


In [37]:

acueductos_por_municipio = df['municipio'].value_counts().reset_index()
acueductos_por_municipio.columns = ['Municipio', 'Número de Acueductos']

fig = px.bar(acueductos_por_municipio,
             x='Municipio',
             y='Número de Acueductos',
             title='Número de Acueductos por Municipio',
             labels={'Municipio': 'Municipio', 'Número de Acueductos': 'Número de Acueductos'})

fig.update_layout(xaxis=dict(tickangle=90))
fig.show()


In [38]:
acueductos_por_provincia = df['provincia'].value_counts().reset_index()
acueductos_por_provincia.columns = ['Provincia', 'Número de Acueductos']

fig = px.bar(acueductos_por_provincia,
             x='Provincia',
             y='Número de Acueductos',
             title='Número de Acueductos por Provincia',
             labels={'Provincia': 'Provincia', 'Número de Acueductos': 'Número de Acueductos'},
             color_discrete_sequence=['orange'])

fig.update_layout(xaxis=dict(tickangle=90))
fig.show()


In [46]:
fuentes_abastecedoras = df['fuente_abastecedora'].value_counts().head(10).reset_index()
fuentes_abastecedoras.columns = ['Fuente Abastecedora', 'Número de Acueductos']

fig = px.bar(fuentes_abastecedoras,
             x='Fuente Abastecedora',
             y='Número de Acueductos',
             title='Fuentes Abastecedoras Más Comunes',
             color='Número de Acueductos',
             color_continuous_scale='Purples',
             labels={'Fuente Abastecedora': 'Fuente Abastecedora', 'Número de Acueductos': 'Número de Acueductos'})

fig.update_layout(xaxis=dict(tickangle=45))
fig.show()
